In [1]:
!pip install selenium

   ---------------------------------------- 0.0/63.8 kB ? eta -:--:--
   ------ --------------------------------- 10.2/63.8 kB ? eta -:--:--
   ------ --------------------------------- 10.2/63.8 kB ? eta -:--:--
   ------ --------------------------------- 10.2/63.8 kB ? eta -:--:--
   -------------------------------------- - 61.4/63.8 kB 365.7 kB/s eta 0:00:01
   ---------------------------------------- 63.8/63.8 kB 382.7 kB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [39]:
!pip install webdriver-manager

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm 

In [29]:
# Path ke chromedriver
chrome_driver_path = r"D:\chromedriver-win64\chromedriver.exe"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

url = "https://sinta.kemdiktisaintek.go.id/authors"
driver.get(url)

In [ ]:
TARGET = 1000
time.sleep(3)

all_links = set()

pbar = tqdm(total=TARGET, desc="Mengumpulkan author links")

while True:
    author_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/authors/profile/']")
    for link in author_links:
        href = link.get_attribute("href")
        if href and href not in all_links:
            all_links.add(href)
            pbar.update(1) 

    if len(all_links) >= TARGET:
        print("Sudah terkumpul 1000 link author, berhenti.")
        break

    try:
        next_button = driver.find_element(By.LINK_TEXT, "Next")
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)
    except:
        print("Tidak ada tombol Next lagi, selesai.")
        break

pbar.close()

for i, l in enumerate(list(all_links)[:TARGET], 1):
    print(i, l)


Mengumpulkan author links: 100%|██████████| 1000/1000 [11:48<00:00,  1.41it/s]


Sudah terkumpul 1000 link author, berhenti.
1 https://sinta.kemdiktisaintek.go.id/authors/profile/5974337
2 https://sinta.kemdiktisaintek.go.id/authors/profile/5975293
3 https://sinta.kemdiktisaintek.go.id/authors/profile/6004215
4 https://sinta.kemdiktisaintek.go.id/authors/profile/6003677
5 https://sinta.kemdiktisaintek.go.id/authors/profile/6000584
6 https://sinta.kemdiktisaintek.go.id/authors/profile/5979634
7 https://sinta.kemdiktisaintek.go.id/authors/profile/257843
8 https://sinta.kemdiktisaintek.go.id/authors/profile/5975888
9 https://sinta.kemdiktisaintek.go.id/authors/profile/6649906
10 https://sinta.kemdiktisaintek.go.id/authors/profile/6028041
11 https://sinta.kemdiktisaintek.go.id/authors/profile/6071022
12 https://sinta.kemdiktisaintek.go.id/authors/profile/6009220
13 https://sinta.kemdiktisaintek.go.id/authors/profile/6903290
14 https://sinta.kemdiktisaintek.go.id/authors/profile/5980523
15 https://sinta.kemdiktisaintek.go.id/authors/profile/6031882
16 https://sinta.kemd

In [13]:
links = list(all_links)

In [15]:
links

['https://sinta.kemdiktisaintek.go.id/authors/profile/5974337',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/5975293',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6004215',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6003677',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6000584',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/5979634',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/257843',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/5975888',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6649906',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6028041',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6071022',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6009220',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6903290',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/5980523',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6031882',
 'https://sinta.kemdiktisaintek.go.id/aut

In [17]:
from IPython.display import FileLink

with open("links.txt", "w") as f:
    for item in links:
        f.write(str(item) + "\n")

FileLink("links.txt")

C:\Users\VICTUS\Downloads\links.txt

In [ ]:
def quick_text(elem):
    return elem.text.strip() if elem else ""

data = []

for i, url in enumerate(tqdm(links, desc="Scraping progress")):
    driver.get(url)
    time.sleep(1.5)  # kasih jeda supaya elemen muncul

    # Nama
    try:
        nama_elem = driver.find_element(By.CSS_SELECTOR, "h3 a")
        nama = quick_text(nama_elem)
    except:
        nama = ""

    # Perguruan Tinggi & Prodi
    try:
        meta_elem = driver.find_element(By.CSS_SELECTOR, ".meta-profile")
        meta_text = meta_elem.text.strip().split("\n")
        perguruan_tinggi = meta_text[0] if len(meta_text) > 0 else ""
        program_studi    = meta_text[1] if len(meta_text) > 1 else ""
    except:
        perguruan_tinggi = ""
        program_studi = ""

    # Bidang
    bidang_elems = driver.find_elements(By.CSS_SELECTOR, ".profile-subject a")
    bidang = ", ".join([quick_text(b) for b in bidang_elems])

    # Sinta Score
    skor_elems = driver.find_elements(By.CSS_SELECTOR, ".stat-profile .pr-num")
    skor = [quick_text(s) for s in skor_elems]
    sinta_overall, sinta_3yr, affil_overall, affil_3yr = (skor + ["", "", "", ""])[:4]

    data.append({
        "url": url,
        "nama": nama,
        "perguruan_tinggi": perguruan_tinggi,
        "program_studi": program_studi,
        "bidang": bidang,
        "sinta_score_overall": sinta_overall,
        "sinta_score_3yr": sinta_3yr,
        "affil_score_overall": affil_overall,
        "affil_score_3yr": affil_3yr
    })

    if i < 3:  # preview 3 data pertama
        print(f"\nData ke-{i+1}: {data[-1]}")

    time.sleep(0.3)

driver.quit()

Scraping progress:   0%|          | 0/1000 [00:00<?, ?it/s]


Data ke-1: {'url': 'https://sinta.kemdiktisaintek.go.id/authors/profile/5974337', 'nama': 'ZETRA HAINUL PUTRA', 'perguruan_tinggi': 'Universitas Riau', 'program_studi': 'S2 - Pendidikan Dasar', 'bidang': 'Mathematics Education, Didactics of Mathematics, Pendidikan Guru Sekolah Dasar, Pendidikan Matematika, Teacher Education', 'sinta_score_overall': '4.280', 'sinta_score_3yr': '1.537', 'affil_score_overall': '4.279', 'affil_score_3yr': '1.537'}


Scraping progress:   0%|          | 1/1000 [00:02<40:32,  2.43s/it]


Data ke-2: {'url': 'https://sinta.kemdiktisaintek.go.id/authors/profile/5975293', 'nama': 'ALFIAN AKBAR GOZALI', 'perguruan_tinggi': 'Universitas Telkom', 'program_studi': 'D3 - Rekayasa Perangkat Lunak Aplikasi', 'bidang': 'Data Analytics, Machine Learning, Optimization Algorithm, Mobile Development', 'sinta_score_overall': '2.814', 'sinta_score_3yr': '1.133', 'affil_score_overall': '120', 'affil_score_3yr': '90'}


Scraping progress:   0%|          | 2/1000 [00:04<39:05,  2.35s/it]


Data ke-3: {'url': 'https://sinta.kemdiktisaintek.go.id/authors/profile/6004215', 'nama': 'DWI SUHARTANTO', 'perguruan_tinggi': 'Politeknik Negeri Bandung', 'program_studi': 'S2 Terapan - Pemasaran, Inovasi, dan Teknologi', 'bidang': 'Marketing, Green Marketing, Tourism Marketing, Islamic Marketing, Consumer behavior', 'sinta_score_overall': '5.419', 'sinta_score_3yr': '1.145', 'affil_score_overall': '0', 'affil_score_3yr': '0'}


Scraping progress:  95%|█████████▌| 954/1000 [37:44<01:50,  2.40s/it]  

In [ ]:
df = pd.DataFrame(data)

df.to_csv("sinta_authors.csv", index=False, encoding="utf-8-sig")
df.to_excel("sinta_authors.xlsx", index=False)
print("Data berhasil disimpan ke 'sinta_authors.csv' dan 'sinta_authors.xlsx'")

In [ ]:
df

In [ ]:
df.isnull().sum()